In [ ]:
import pandas as pd
import datetime
import yfinance as yf

### 0. 필요 파일 목록

*  'nst_{fish}_trend_2025-01-17.csv'
*  'forecast_agg.csv'
* 'item_price_lag_filled.csv'

### 1. 경제지표 데이터 전처리

filled_economic_indicators.csv 생성

#### 1-1. yfinance에서 데이터 불러오기

* 입력 파일 :  없음
* 출력 파일 : '_economic_indicators_.csv'

In [ ]:
import pandas as pd
import datetime
import yfinance as yf

# 시작일과 종료일 설정
start_date = '2015-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

# 데이터를 저장할 빈 데이터프레임 생성
df_final = pd.DataFrame()

# 각 지표의 티커 심볼 정의
tickers = {
    'KOSPI': '^KS11',  # KOSPI
    'USD/KRW': 'KRW=X',  # 원달러 환율
    'WTI': 'CL=F',  # WTI 원유 선물
    'VIX': '^VIX',  # VIX 지수
    'Gold': 'GC=F',  # 금 선물
    'Silver': 'SI=F',  # 은 선물
    'MOVE' : '^MOVE' # MOVE Index
}

# 각 티커에 대해 데이터 다운로드
for name, ticker in tickers.items():
    try:
        df = yf.download(ticker, start=start_date, end=end_date)
        # 종가만 사용
        df_final[name] = df['Close']
    except Exception as e:
        print(f"Error downloading {name}: {e}")

# 결측치 처리
df_final = df_final.fillna(method='ffill')

# 데이터 확인
print("\nFirst few rows of the data:")
print(df_final.head())

# 기본 통계 확인
print("\nBasic statistics:")
print(df_final.describe())

# CSV 파일로 저장
output_filename = '_economic_indicators_.csv'
df_final.to_csv(output_filename)
print(f"\nData saved to {output_filename}")

#### 1-2. 경제지표 데이터 결측치 채우기 

* 입력 파일 : '_economic_indicators_.csv'
* 출력 파일 : 'filled_economic_indicators_.csv'

In [ ]:
import pandas as pd
import datetime

# 데이터 로드
df = pd.read_csv('_economic_indicators_.csv', encoding='utf-8')

# Date 컬럼을 datetime으로 변환
df['Date'] = pd.to_datetime(df['Date'])

# 시작일과 마지막일 추출
start_date = df['Date'].min()
end_date = df['Date'].max()

# 모든 날짜가 포함된 데이터프레임 생성
all_dates = pd.DataFrame(
    {'Date': pd.date_range(start_date, end_date, freq='D')}
)

# 기존 데이터와 병합
filled_df = pd.merge(all_dates, df, on='Date', how='left')

# 결측치를 이전 값으로 채우기
filled_df = filled_df.ffill()

# 결과 저장
filled_df.to_csv('filled_economic_indicators.csv', index=False)

# 결과 확인
print("처리된 데이터 샘플:")
print(filled_df.head())
print("\n결측치 확인:")
print(filled_df.isnull().sum())

### 2. 트렌드 데이터 어종별로 그룹화



#### 2-1. 그룹화 함수 생성 및 저장

* 입력 파일 : 'nst_{fish}_trend_2025-01-17.csv'
* 출력 파일 : '그룹화_nst_{fish}_trend_2025-01-17.csv'

In [ ]:
import pandas as pd

def process_age_groups(file_path):
    # CSV 파일 읽기
    df = pd.read_csv(file_path, encoding='utf-8')
    
    # 연령대 그룹 매핑 딕셔너리 생성
    age_mapping = {
        '19_24': '20대',
        '25_29': '20대',
        '30_34': '30대',
        '35_39': '30대',
        '40_44': '40대',
        '45_49': '40대',
        '50_54': '50대',
        '55_59': '50대',
        '60_80': '60대 이상'
    }
    
    # 원하는 연령대만 필터링
    df = df[df['age'].isin(age_mapping.keys())]
    
    # 새로운 연령대 컬럼 생성
    df['age_group'] = df['age'].map(age_mapping)
    
    # 일자별, 새로운 연령대별로 score 합산
    result = df.groupby(['date', 'name', 'age_group'])['score'].sum().reset_index()
    
    # 피벗 테이블로 변환하여 보기 좋게 정리
    pivot_result = result.pivot(index=['date', 'name'], 
                              columns='age_group', 
                              values='score').reset_index()
    
    # 컬럼 순서 정리
    column_order = [ 'date', 'name', '20대', '30대', '40대', '50대', '60대 이상']
    pivot_result = pivot_result[column_order]
    
    return pivot_result



In [ ]:
#  수산물 종류별 파일 경로 및 이름 설정
fish_files = {
    '광어': '../../data/raw/nst_광어_trend_2025-01-17.csv',
    '농어': '../../data/raw/nst_농어_trend_2025-01-17.csv',
    '대게': '../../data/raw/nst_대게_trend_2025-01-17.csv',
    '방어': '../../data/raw/nst_방어_trend_2025-01-17.csv',
    '연어': '../../data/raw/nst_연어_trend_2025-01-17.csv',
    '우럭': '../../data/raw/nst_우럭_trend_2025-01-17.csv',
    '참돔': '../../data/raw/nst_참돔_trend_2025-01-17.csv'
}

#  개별 파일 저장 + 결과 리스트로 출력 준비
all_results = {}

#  모든 파일 처리
for fish, file_path in fish_files.items():
    try:
        processed_df = process_age_groups(file_path, fish)

        # 개별 파일 저장
        output_filename = f'그룹화_nst_{fish}_trend_2025-01-17.csv'
        processed_df.to_csv(output_filename, index=False)

        # 결과 리스트에 추가 (첫 3개 행만 저장)
        all_results[fish] = processed_df.head(3)

        print(f"✅ '{output_filename}' 저장 완료!")

    except Exception as e:
        print(f"❌ Error processing {fish}: {e}")

#  최종 처리된 결과 출력 (각각 3개 행만 출력)
print("\n📊 최종 처리된 결과 미리보기 (각각 상위 3개 행)\n")
for fish, df_sample in all_results.items():
    print(f"{fish} 데이터 샘플:")
    print(df_sample)
    print("-" * 50)  # 구분선 추가

#### 2-2. merged_trends.csv 저장

* 입력 파일 : '그룹화_nst_{fish}_trend_2025-01-17.csv'
* 출력 파일 : 'merged_trends.csv'

In [ ]:
def merge_fish_trends(file_paths, output_path):
   # 첫 번째 파일 로드 및 name 컬럼 제거
   merged_df = pd.read_csv(file_paths[0], encoding='utf-8')
   merged_df = merged_df.drop('name', axis=1)
   merged_df['date'] = pd.to_datetime(merged_df['date'])
   
   # 나머지 파일들 병합
   for file_path in file_paths[1:]:
       df = pd.read_csv(file_path, encoding='utf-8')
       df = df.drop('name', axis=1)
       df['date'] = pd.to_datetime(df['date'])
       merged_df = pd.merge(merged_df, df, on='date', how='outer')
   
   # 날짜순 정렬
   result = merged_df.sort_values('date')
   
   # CSV 저장
   result.to_csv(output_path, index=False, encoding='utf-8')
   return result


In [ ]:
# 사용 예시:
files = [
    '그룹화_nst_광어_trend_2025-01-17.csv', 
    '그룹화_nst_농어_trend_2025-01-17.csv', 
    '그룹화_nst_대게_trend_2025-01-17.csv', 
    '그룹화_nst_방어_trend_2025-01-17.csv', 
    '그룹화_nst_연어_trend_2025-01-17.csv', 
    '그룹화_nst_우럭_trend_2025-01-17.csv', 
    '그룹화_nst_참돔_trend_2025-01-17.csv'
    ]

result = merge_fish_trends(files, 'merged_trends.csv')

### 3. 기상 데이터 전처리

* 입력 파일 : 'forecast_agg.csv'
* 출력 파일 : 'weatherdata_processed.csv'


In [1]:
# 함수 정의

def create_weather_columns(df, station_cols, output_path):
   result_df = df.copy()
   result_df['일시'] = pd.to_datetime(result_df['일시'])
   
   final_df = pd.DataFrame({'일시': result_df['일시'].unique()})
   
   for station, columns in station_cols.items():
       station_data = result_df[result_df['지점'] == station].copy()
       
       for orig_col, new_col in columns:
           station_col = station_data[['일시', orig_col]].copy()
           final_df = pd.merge(final_df, station_col.rename(columns={orig_col: new_col}), 
                             on='일시', how='left')
   
   # 칼럼을 가나다순으로 정렬 ('일시' 컬럼은 첫번째로 유지)
   sorted_cols = ['일시'] + sorted([col for col in final_df.columns if col != '일시'])
   result = final_df[sorted_cols].sort_values('일시')
   
   result.to_csv(output_path, index=False, encoding='utf-8')
   return result

In [ ]:
# CSV 파일 읽기
df = pd.read_csv('forecast_agg.csv', encoding='utf-8')

# 피쳐 선택
station_columns = {
	22105: [
		('기온', '광어_기온_22105'),
		('기온', '농어_기온_22105'),
		('기온', '대게_기온_22105'),
		('파주기', '대게_파주기_22105'),
		('파주기', '방어_파주기_22105'),
		('기온', '연어_기온_22105'),
		('습도', '연어_습도_22105')
	],
    
	22107: [
		('수온', '광어_수온_22107'),
		('수온', '농어_수온_22107'),
		('수온', '방어_수온_22107'),
		('수온', '연어_수온_22107'),
		('수온', '참돔_수온_22107')
	],

	22186: [
		('습도', '광어_습도_22186'),
		('습도', '농어_습도_22186'),
		('기온', '우럭_기온_22186'),
		('수온', '우럭_수온_22186')
		],
        
	22188: [
		('수온', '대게_수온_22188'),
		('습도', '대게_습도_22188')
		],        

	22189: [
		('파주기', '우럭_파주기_22189')
		],       

	22190: [
		('파주기', '광어_파주기_22190'),
        ('파주기', '농어_파주기_22190'),
        ('기온', '방어_기온_22190'),
        ('습도', '방어_습도_22190'),
        ('파주기', '연어_파주기_22190'),
        ('습도', '우럭_습도_22190'),
        ('기온', '참돔_기온_22190'),
        ('습도', '참돔_습도_22190'),
        ('파주기', '참돔_파주기_22190')
		]  

	}

result = create_weather_columns(df, station_columns, 'weatherdata_processed.csv')

### 4. 변수 하나로 합치기

* 입력 파일 : 'filled_economic_indicators_.csv',  'merged_trends.csv', 'weatherdata_processed.csv'
* 출력 파일 : 'merged_all_data.csv'

In [ ]:
def merge_all_data(output_path='merged_all_data.csv'):
   # 각 파일 로드 및 전처리
   
   economic = pd.read_csv('_filled_economic_indicators.csv', encoding='utf-8')
   economic['Date'] = pd.to_datetime(economic['Date'])
   economic = economic.rename(columns={'Date': '날짜'})
   
   trends = pd.read_csv('merged_trends.csv', encoding='utf-8')
   trends['date'] = pd.to_datetime(trends['date'])
   trends = trends.rename(columns={'date': '날짜'})
   
   weather = pd.read_csv('weatherdata_processed.csv', encoding='utf-8')
   weather['date'] = pd.to_datetime(weather['date'])
   weather = weather.rename(columns={'date': '날짜'})
   
   # 모든 날짜 추출
   all_dates = pd.concat([
        economic['날짜'], 
        trends['날짜'], 
       weather['날짜']
   ]).unique()
   
   # 날짜 기준 데이터프레임 생성
   date_df = pd.DataFrame({'날짜': all_dates})
   date_df = date_df.sort_values('날짜')
   
   # 데이터 병합
   dfs = [
		date_df, 
		economic, 
		trends,
		weather
   ]
   
   result = dfs[0]
   for df in dfs[1:]:
       result = pd.merge(result, df, on='날짜', how='left')
   
   # 결과 저장
   result.to_csv(output_path, index=False, encoding='utf-8')
   return result


In [ ]:
result = merge_all_data()

### 5. 결측치 채우기

* 입력 파일 : 'merged_all_data.csv'
* 출력 파일 : 'filled_merged_all_data.csv'

In [ ]:
import pandas as pd

def fill_missing_values(file_path, output_path):
    """
    CSV 파일에서 결측치를 `ffill`을 사용하여 채우되, 이전 값이 없으면 그대로 둠.
    
    Args:
        file_path (str): 입력 CSV 파일 경로.
        output_path (str): 처리된 데이터를 저장할 경로.
    
    Returns:
        pd.DataFrame: 결측치가 채워진 데이터프레임.
    """
    
    # CSV 파일 로드
    df = pd.read_csv(file_path)

    # 결측치 확인 (처리 전)
    missing_before = df.isnull().sum()

    # 결측치 `ffill`로 채우기 (이전 값이 없으면 그대로 둠)
    df.fillna(method='ffill', inplace=True)

    # 결측치 확인 (처리 후)
    missing_after = df.isnull().sum()

    # 변경된 결측치 개수 출력
    missing_summary = pd.DataFrame({'Before': missing_before, 'After': missing_after})
    print("\n🔍 결측치 처리 전후 비교:")
    print(missing_summary)

    # 처리된 데이터 저장
    df.to_csv(output_path, index=False)
    print(f"\n✅ 처리된 데이터가 '{output_path}'에 저장되었습니다.")

    return df




In [ ]:
file_path = 'merged_all_data.csv'
output_path = 'filled_merged_all_data.csv'

fill_missing_values(file_path, output_path)


### 6. 타임래그 적용하기 

* 입력 파일 : 'filled_merged_all_data.csv'
* 출력 파일 : 'timelagged_features.csv'

#### 6-1. 설정된 타임래그 적용

In [ ]:
# 데이터 로드
df = pd.read_csv('filled_merged_all_data.csv', encoding='utf-8')
df['date'] = pd.to_datetime(df['날짜'])

# 각 칼럼별 lag 일수 정의
lag_days = {

'연어_거래량(톤)' : 374,
'연어_가격(NOK/kg)' : 32,

'광어_대'  : 64,
'방어(자연)_대_가락' : 5,
'참돔_대_가락' : 1,

'광어_KOSPI' : 136,
'광어_USD/KRW' : 1,
'광어_WTI' : 1,
'광어_VIX' : 399,
'광어_Gold' : 314,
'광어_Silver' : 238,
'광어_MOVE' : 18,

'농어_KOSPI' : 179,
'농어_USD/KRW' : 1,
'농어_WTI' : 100,
'농어_VIX' : 391,
'농어_Gold' : 361,
'농어_Silver' : 290,
'농어_MOVE' : 1,

'대게_KOSPI' : 148,
'대게_USD/KRW' : 90,
'대게_WTI' : 91,
# '대게_VIX' : 
'대게_Gold' : 177,
'대게_Silver' : 177,
# '대게_MOVE' : 

'방어_KOSPI' : 282,
'방어_USD/KRW' : 387,
'방어_WTI' : 399,
'방어_VIX' : 133,
'방어_Gold' : 1,
'방어_Silver' : 1,
'방어_MOVE' : 381,

'연어_KOSPI' : 329,
'연어_USD/KRW' : 1,
'연어_WTI' : 199,
'연어_VIX' : 399,
'연어_Gold' : 1,
'연어_Silver' : 399,
'연어_MOVE' : 71,

'우럭_KOSPI' : 155,
'우럭_USD/KRW' : 121,
'우럭_WTI' : 14,
'우럭_VIX' : 38,
'우럭_Gold' : 146,
'우럭_Silver' : 125,
# '우럭_MOVE' : 

'참돔_KOSPI' : 399,
'참돔_USD/KRW' : 11,
'참돔_WTI' : 150,
# '참돔_VIX' : 
# '참돔_Gold' : 
# '참돔_Silver' : 
'참돔_MOVE' : 201,
 
'광어_20대' : 250,
'광어_30대' : 317,
'광어_40대' : 330,
'광어_50대' : 395,
'광어_60대이상' : 339,

'농어_20대' : 305,
'농어_30대' : 307,
'농어_40대' : 309,
'농어_50대' : 309,
'농어_60대이상' : 273,

'대게_20대' : 273,
'대게_30대' : 370,
'대게_40대' : 5,
'대게_50대' : 5,
'대게_60대이상' : 5,

'방어_20대' : 22,
'방어_30대' : 26,
'방어_40대' : 256,
'방어_50대' : 256,
'방어_60대이상' : 26,

'연어_20대' : 20,
'연어_30대' : 15,
'연어_40대' : 21,
'연어_50대' : 15,
'연어_60대이상' : 397,

'우럭_20대' : 22,
'우럭_30대' : 354,
'우럭_40대' : 354,
'우럭_50대' : 220,
'우럭_60대이상' : 219,

'참돔_20대' : 197,
'참돔_30대' : 167,
'참돔_40대' : 278,
'참돔_50대' : 193,
'참돔_60대이상' : 14,


'광어_기온_22105' : 97,
'광어_수온_22107' : 79,
'광어_습도_22186' : 349,
'광어_파주기_22190' : 103,

'농어_기온_22105' : 103,
'농어_수온_22107' : 81,
'농어_습도_22186' : 333,
'농어_파주기_22190' : 115,

'대게_기온_22105' : 150,
'대게_수온_22188' : 140,
'대게_습도_22188' : 355,
'대게_파주기_22105' : 174,

'방어_기온_22190' : 114,
'방어_수온_22107' : 118,
'방어_습도_22190' : 158,
'방어_파주기_22105' : 191,

'연어_기온_22105' : 394,
'연어_수온_22107' : 341,
'연어_습도_22105' : 8,
'연어_파주기_22190' : 9,

'우럭_기온_22186' : 118,
'우럭_수온_22186' : 113,
'우럭_습도_22190' : 159,
'우럭_파주기_22189' : 172,

'참돔_기온_22190' : 91,
'참돔_수온_22107' : 72,
'참돔_습도_22190' : 115,
'참돔_파주기_22190' : 107


}

# 각 칼럼에 대해 지정된 lag 적용
for col, lag in lag_days.items():
   if col in df.columns:
       df[f'{col}_{lag}'] = df[col].shift(lag)

# 결과 저장
df.to_csv('timelagged_features.csv', index=False, encoding='utf-8')

#### 6-2 트렌드 데이터 lag 1 합치기

### 7. 가격 데이터 - 노량진 2층 버리기

* 입력 파일 : 'item_price_lag_filled.csv'
* 출력 파일 : 'item_price_lag_filled_deleted.csv'

In [ ]:
import pandas as pd

# 파일 경로 설정
file_path = "item_price_lag_filled.csv"  # 원본 데이터 파일
output_file = "item_price_lag_filled_deleted.csv"  # 저장될 파일

# 데이터 로드
df = pd.read_csv(file_path)

# 'market' 칼럼에서 '노량진 2층'인 행 제거
df = df[df['market'] != '노량진 2층']

# '노량진 1층'을 '노량진시장'으로 변경
df['market'] = df['market'].replace('노량진 1층', '노량진시장')

# 처리된 데이터 저장
df.to_csv(output_file, index=False)

# 변경된 데이터 확인
print("\n✅ 변경된 데이터 (상위 5개 행):")
print(df.head())

print(f"\n📁 처리된 파일이 '{output_file}'로 저장되었습니다!")


### 8. 시장 데이터 원핫 인코딩

* 입력 파일 : 'item_price_lag_filled_deleted.csv'
* 출력 파일 : 'item_price_oneHot.csv'

In [ ]:
import pandas as pd
import DateTime

def transform_market_data(file_path):
    # 데이터 읽기
    df = pd.read_csv(file_path)
    
    # 시장별 더미변수 생성 (0, 1로 인코딩)
    market_dummies = pd.get_dummies(df['market'], prefix='m').astype(int)
    
    # 원본 데이터와 더미변수 결합
    result = pd.concat([
        df[['priceDate', 'item']], 
        market_dummies,
        df[['avgPrice', 'avgPrice_lag_1']]
    ], axis=1)
    
    # 날짜와 어종으로 정렬
    result = result.sort_values(['priceDate', 'item'])
    
    # 변환된 데이터 저장
    output_file = 'item_price_oneHot.csv'
    result.to_csv(output_file, index=False)
    print(f"생성된 파일: {output_file}")
    print("\n처음 10개 컬럼:", result.columns[:10].tolist())
    
    return result



In [ ]:
# Execute
df = transform_market_data('item_price_lag_filled_deleted.csv')

생성된 파일: transformed_market_data.csv

처음 10개 컬럼: ['priceDate', 'item', 'm_가락시장', 'm_강서농수산물시장', 'm_구리농수산물시장', 'm_노량진 1층', 'm_노량진 2층', 'm_마포농수산물시장', 'm_부산민락어민활어직판장', 'm_소래포구종합어시장']


### 9. 가격데이터 어종별로 나누기

* 입력 파일 : 'item_price_oneHot.csv'
* 출력 파일 : '{fish}_price_oneHot.csv'

In [ ]:
import pandas as pd

def split_market_data():
    df = pd.read_csv('item_price_oneHot.csv')
    
    for fish in df['item'].unique():
        fish_df = df[df['item'] == fish]
        output_file = f'{fish}_price_oneHot.csv'
      # fish_df = fish_df.drop('item', axis=1)
        fish_df.to_csv(output_file, index=False)
        print(f'{fish} 데이터 생성 완료: {len(fish_df)}개 행')



In [ ]:
split_market_data()

대게 데이터 생성 완료: 27677개 행
광어 데이터 생성 완료: 31515개 행
농어 데이터 생성 완료: 30651개 행
연어 데이터 생성 완료: 29588개 행
참돔 데이터 생성 완료: 27149개 행
방어 데이터 생성 완료: 11205개 행
우럭 데이터 생성 완료: 6081개 행


### 10. 최종 데이터 합치기

* 입력 파일 : '{fish}_price_oneHot.csv', '{fish}_timelagged_features.csv'
* 출력 파일 : '{fish}_price_features.csv'

In [ ]:
import pandas as pd

def merge_features_price():
    # 두 데이터셋 로드
    features_df = pd.read_csv('../../data/features/참돔_timelagged_features.csv')
    price_df = pd.read_csv('../../data/features/oneHot/참돔_price_oneHot.csv')
    
    # 날짜 컬럼명 통일
    price_df = price_df.rename(columns={'dmdpriceDate': 'date'})
    
    # 날짜 기준으로 데이터 병합
    merged_df = pd.merge(price_df, features_df, on='date', how='left')
    
    # CSV 파일로 저장
    output_file = '참돔_price_features.csv'
    merged_df.to_csv(output_file, index=False)
    print(f'생성된 파일: {output_file}')
    print(f'전체 컬럼 수: {len(merged_df.columns)}')

merge_features_price()

생성된 파일: 참돔_price_features.csv
전체 컬럼 수: 35
